# Report 05. Jeun Hyun (Student ID -20191328)


In [21]:
# The function to read sequences from a FASTA file
def read_fasta(tmp_filename):
    rv = dict()
    f = open(tmp_filename, 'r')
    for line in f:
        if line.startswith('>'):
            tmp_h = line.strip().lstrip('>')
            rv[tmp_h] = ''
        else:
            rv[tmp_h] += line.strip().replace(' ', '')
    f.close()
    return rv

In [22]:
# make clean process
genome = read_fasta('2019nCoV_genomes.2020_04_14.clean.fa')
for i in genome.keys():
    if len(i.split('|')[2]) == 4:
        genome[i+'-00-00'] = genome[i]
        break

n_genome = {}
for i in genome.keys():
    n_genome[i.replace('|',';')] = genome[i]

f = open("../data/2019nCoV_genomes.2020_04_14.clean_n.fa", 'w')
for name in n_genome.keys():
    data = "> %s\n%s\n" %(name, n_genome[name])
    f.write(data)
f.close()

In [23]:
# Read GCF_009858895.2 genomic info.
filename_genomes = '../data/GCF_009858895.2_ASM985889v3_genomic.fna'
genome_list = read_fasta(filename_genomes)
ref_seq = list(genome_list.values())[0]

In [24]:
#Make ref.orf1ab cDNA sequence.
f = open('../data/GCF_009858895.2_ASM985889v3_genomic.gff', 'r')
count = 1
gene_dict = {}
for line in f:
    try:
        if 'CDS' in line.split('	')[2] and 'product=orf1ab polyprotein' in line.split('	')[8]:
            gene_dict['orf1ab_polyprotein_%s'%count] = ref_seq[int(line.split()[3])-1:int(line.split()[4])]
            count +=1
    except:
        continue        
f.close()

# Make other CDN's cDNA ref.sequences.
f = open('../data/GCF_009858895.2_ASM985889v3_genomic.gff', 'r')
for line in f:
    #print('line')
    try:
        #print('try')
        if 'CDS' == line.split('	')[2]:
            info = line.split('	')[8].split(';')
            for h in info:
                if 'product=' in h:
                    name = h[8:]
                    name_genomic = ref_seq[int(line.split()[3])-1 : int(line.split()[4])]
                    #print('cDNA sequence of \'%s gene\' is'%name, name_genomic[:25], '...', name_genomic[-25:])
                    if name != 'orf1ab polyprotein':
                        gene_dict[name.replace(' ','_')] = name_genomic
    except:
        continue
#print(list(gene_dict.keys()))
f.close()




f = open("../data/gene_dict.fa", 'w')
for name in gene_dict.keys():
    #print(name)
    data = "> %s\n%s\n" %(name, gene_dict[name])
    f.write(data)
f.close()


# Do transeq and get reference protein sequences of each gene
# Do blastp and get the location of protein sequences of subject fit into the reference sequence
# (terminal) ~/git/BME203-2020$ transeq --sequence ./data/gene_dict.fa --outseq ./data/gene_dict_trans.fa
# (terminal) ~/git/BME203-2020$ transeq --sequence ./data/2019nCoV_genomes.2020_04_14.clean_n.fa --outseq ./data/2019nCoV_genomes.2020_04_14.clean_n_transF.fa -frame F
# (terminal) ~/git/BME203-2020$ makeblastdb -dbtype prot -in ./data/2019nCoV_genomes.2020_04_14.clean_n_transF.fa -out ./data/2019nCoV_genomes.2020_04_14.clean_n_transF
# (terminal) ~/git/BME203-2020$ blastp -db ./data/2019nCoV_genomes.2020_04_14.clean_n_transF -query ./data/gene_dict_trans.fa -outfmt 6 -out ./data/2019nCoV_genomes.2020_04_14.clean_n_transF_blastp.out -max_target_seqs 100000 -max_hsps 1

In [25]:
# extract the protein of each genes from each subjects.
query_list = ['orf1ab_polyprotein_query','orf1a_polyprotein_query', 'surface_glycoprotein_query', 'ORF3a_protein_query', 'envelope_protein_query', 'membrane_glycoprotein_query' ,'ORF6_protein_query' ,'ORF7a_protein_query' ,'ORF7b_query', 'ORF8_protein_query', 'nucleocapsid_phosphoprotein_query', 'ORF10_protein_query']

transF_list = read_fasta('../data/2019nCoV_genomes.2020_04_14.clean_n_transF.fa')
temp = ()

for query in query_list:
    f = open('../data/2019nCoV_genomes.2020_04_14.clean_n_transF_blastp.out', 'r')
    #print('query_list')
    query_dict = {}
    for line in f:
        sep = line.split('	')
        #print(sep[0].split('_')[0], query)
        if (sep[0],sep[1][:-2])!= temp:
            if sep[0].split('_')[0] in query:
                #print(sep[1][:-2],len(query_dict.keys()))
                p_s = int(sep[8])-1
                p_l = int(sep[9])
                if sep[0][:-2] == 'orf1ab_polyprotein_1':
                    query_dict[sep[1][:-2]] = transF_list[sep[1]][p_s:p_l]
                    #print(p_s, p_l)
                elif sep[0][:-2] == 'orf1ab_polyprotein_2':
                    query_dict[sep[1][:-2]] = query_dict[sep[1][:-2]] + transF_list[sep[1]][p_s:p_l]
                    #print(query_dict[sep[1]][:25])
                else:
                    query_dict[sep[1][:-2]] = transF_list[sep[1]][p_s:p_l]
                    #print(query_dict[sep[1]][:25])
                temp = (sep[0],sep[1][:-2])
    

    h = open("../data/%s_protein_dict.fa"%query, 'w')
    for name in query_dict.keys():
        #print(name)
        data = "> %s\n%s\n" %(name, query_dict[name])
        h.write(data)
    h.close() 
    f.close()

# Do muscle and get the seuqnces matched amino acids with the same length
# (terminal) ~/git/BME203-2020$ muscle -in ./data/orf1ab_polyprotein_query_protein_dict.fa -out ./data/orf1ab_polyprotein_query_protein_dict_muscle.fa
# (terminal) ~/git/BME203-2020$ muscle -in ./data/orf1a_polyprotein_query_protein_dict.fa -out ./data/orf1a_polyprotein_query_protein_dict_muscle.fa
# (terminal) ~/git/BME203-2020$ muscle -in ./data/surface_glycoprotein_query_protein_dict.fa -out ./data/surface_glycoprotein_query_protein_dict_muscle.fa
# (terminal) ~/git/BME203-2020$ muscle -in ./data/ORF3a_protein_query_protein_dict.fa -out ./data/ORF3a_protein_query_protein_dict_muscle.fa
# (terminal) ~/git/BME203-2020$ muscle -in ./data/envelope_protein_query_protein_dict.fa -out ./data/envelope_protein_query_protein_dict_muscle.fa
# (terminal) ~/git/BME203-2020$ muscle -in ./data/membrane_glycoprotein_query_protein_dict.fa -out ./data/membrane_glycoprotein_query_protein_dict_muscle.fa
# (terminal) ~/git/BME203-2020$ muscle -in ./data/ORF6_protein_query_protein_dict.fa -out ./data/ORF6_protein_query_protein_dict_muscle.fa
# (terminal) ~/git/BME203-2020$ muscle -in ./data/ORF7a_protein_query_protein_dict.fa -out ./data/ORF7a_protein_query_protein_dict_muscle.fa
# (terminal) ~/git/BME203-2020$ muscle -in ./data/ORF7b_query_protein_dict.fa -out ./data/ORF7b_query_protein_dict_muscle.fa
# (terminal) ~/git/BME203-2020$ muscle -in ./data/ORF8_protein_query_protein_dict.fa -out ./data/ORF8_protein_query_protein_dict_muscle.fa
# (terminal) ~/git/BME203-2020$ muscle -in ./data/nucleocapsid_phosphoprotein_query_protein_dict.fa -out ./data/nucleocapsid_phosphoprotein_query_protein_dict_muscle.fa
# (terminal) ~/git/BME203-2020$ muscle -in ./data/ORF10_protein_query_protein_dict.fa -out ./data/ORF10_protein_query_protein_dict_muscle.fa


In [26]:
# extract the cDNA of each genes from each subjects.
query_list = ['orf1ab_polyprotein_query','orf1a_polyprotein_query', 'surface_glycoprotein_query', 'ORF3a_protein_query', 'envelope_protein_query', 'membrane_glycoprotein_query' ,'ORF6_protein_query' ,'ORF7a_protein_query' ,'ORF7b_query', 'ORF8_protein_query', 'nucleocapsid_phosphoprotein_query', 'ORF10_protein_query']
cDNA_list = read_fasta('../data/2019nCoV_genomes.2020_04_14.clean_n.fa')

temp = ()
for query in query_list:
    f = open('../data/2019nCoV_genomes.2020_04_14.clean_n_transF_blastp.out', 'r')
    query_dict = {}
    for line in f:
        sep = line.split('	')
        if (sep[0],sep[1][:-2])!= temp:
            if sep[0].split('_')[0] in query:
                #print(sep[1][:-2],len(query_dict.keys()))
                p_s = int(sep[8])-1
                p_l = int(sep[9])
                if line.split()[1][-1:] == '1':
                    if sep[0][:-2] == 'orf1ab_polyprotein_1':
                        #print('_1')
                        query_dict[sep[1][:-2]] = cDNA_list[' '+sep[1][:-2]][(int(p_s))*3 : int(p_l)*3]
                        #print(int(p_s), (int(p_s))*3, int(p_l)*3, ' '+sep[1][:-2], len(query_dict[sep[1][:-2]]))
                    elif sep[0][:-2] == 'orf1ab_polyprotein_2':
                        #print('_2')
                        query_dict[sep[1][:-2]] = query_dict[sep[1][:-2]] + cDNA_list[' '+sep[1][:-2]][(int(p_s))*3 : int(p_l)*3]
                    else:
                        #print('good')
                        query_dict[sep[1][:-2]] = cDNA_list[' '+sep[1][:-2]][(int(p_s))*3 : int(p_l)*3]
                elif line.split()[1][-1:] == '2':
                    if sep[0][:-2] == 'orf1ab_polyprotein_1':
                        #print('_1')
                        query_dict[sep[1][:-2]] = cDNA_list[' '+sep[1][:-2]][(int(p_s))*3 : int(p_l)*3]
                        #print(int(p_s), (int(p_s))*3, int(p_l)*3, ' '+sep[1][:-2], len(query_dict[sep[1][:-2]]))
                    elif sep[0][:-2] == 'orf1ab_polyprotein_2':
                        #print('_2')
                        query_dict[sep[1][:-2]] = query_dict[sep[1][:-2]] + cDNA_list[' '+sep[1][:-2]][(int(p_s))*3 : int(p_l)*3]
                    else:
                        #print('good')
                        query_dict[sep[1][:-2]] = cDNA_list[' '+sep[1][:-2]][(int(p_s))*3 : int(p_l)*3]
                elif line.split()[1][-1:] == '3':
                    
                    if sep[0][:-2] == 'orf1ab_polyprotein_1':
                        query_dict[sep[1][:-2]] = cDNA_list[' '+sep[1][:-2]][(int(p_s))*3 : int(p_l)*3]
                        #print('_1')
                        #print(int(p_s), (int(p_s))*3, int(p_l)*3, ' '+sep[1][:-2], len(query_dict[sep[1][:-2]]))
                    elif sep[0][:-2] == 'orf1ab_polyprotein_2':
                        #print('_2')
                        query_dict[sep[1][:-2]] = query_dict[sep[1][:-2]] + cDNA_list[' '+sep[1][:-2]][(int(p_s))*3 : int(p_l)*3]
                    else:
                        #print('good')
                        query_dict[sep[1][:-2]] = cDNA_list[' '+sep[1][:-2]][(int(p_s))*3 : int(p_l)*3]
                temp = (sep[0],sep[1][:-2])
    
    h = open("../data/%s_cDNA_dict.fa"%query, 'w')
    for name in query_dict.keys():
        #print(name)
        data = "> %s\n%s\n" %(name, query_dict[name])
        h.write(data)
    h.close() 
    f.close()
    
# Do muscle and get the seuqnces matched nucleotides with the same length  
# (terminal) ~/git/BME203-2020$ muscle -in ./data/orf1ab_polyprotein_query_cDNA_dict.fa -out ./data/orf1ab_polyprotein_query_cDNA_dict_muscle.fa 
# (terminal) ~/git/BME203-2020$ muscle -in ./data/orf1a_polyprotein_query_cDNA_dict.fa -out ./data/orf1a_polyprotein_query_cDNA_dict_muscle.fa  ### ignored
# (terminal) ~/git/BME203-2020$ muscle -in ./data/surface_glycoprotein_query_cDNA_dict.fa -out ./data/surface_glycoprotein_query_cDNA_dict_muscle.fa
# (terminal) ~/git/BME203-2020$ muscle -in ./data/ORF3a_protein_query_cDNA_dict.fa -out ./data/ORF3a_protein_query_cDNA_dict_muscle.fa
# (terminal) ~/git/BME203-2020$ muscle -in ./data/envelope_protein_query_cDNA_dict.fa -out ./data/envelope_protein_query_cDNA_dict_muscle.fa
# (terminal) ~/git/BME203-2020$ muscle -in ./data/membrane_glycoprotein_query_cDNA_dict.fa -out ./data/membrane_glycoprotein_query_cDNA_dict_muscle.fa
# (terminal) ~/git/BME203-2020$ muscle -in ./data/ORF6_protein_query_cDNA_dict.fa -out ./data/ORF6_protein_query_cDNA_dict_muscle.fa
# (terminal) ~/git/BME203-2020$ muscle -in ./data/ORF7a_protein_query_cDNA_dict.fa -out ./data/ORF7a_protein_query_cDNA_dict_muscle.fa
# (terminal) ~/git/BME203-2020$ muscle -in ./data/ORF7b_query_cDNA_dict.fa -out ./data/ORF7b_query_cDNA_dict_muscle.fa
# (terminal) ~/git/BME203-2020$ muscle -in ./data/ORF8_protein_query_cDNA_dict.fa -out ./data/ORF8_protein_query_cDNA_dict_muscle.fa
# (terminal) ~/git/BME203-2020$ muscle -in ./data/nucleocapsid_phosphoprotein_query_cDNA_dict.fa -out ./data/nucleocapsid_phosphoprotein_query_cDNA_dict_muscle.fa
# (terminal) ~/git/BME203-2020$ muscle -in ./data/ORF10_protein_query_cDNA_dict.fa -out ./data/ORF10_protein_query_cDNA_dict_muscle.fa
# fasten processing time : -maxiters 3

In [27]:
# join each genes into one as one corresponding object.(protein)
query_list = ['orf1ab_polyprotein_query', 'surface_glycoprotein_query', 'ORF3a_protein_query', 'envelope_protein_query', 'membrane_glycoprotein_query' ,'ORF6_protein_query' ,'ORF7a_protein_query' ,'ORF7b_query', 'ORF8_protein_query', 'nucleocapsid_phosphoprotein_query', 'ORF10_protein_query']
p_set = {}
for query in query_list:
    dic = read_fasta('../data/%s_protein_dict_muscle.fa'%query)
    for i in range(len(dic.keys())):
        key = list(dic.keys())[i]
        if key in p_set.keys():
            p_set[key] = p_set[key] + dic[key]
            #print(p_set[key][:25])
        else:
            p_set[key] = dic[key]



h = open("../data/SARS_CoV_N_protein_sequence.muscle.fa", 'w')
for name in p_set.keys():
    data = "> %s\n%s\n" %(name, p_set[name])
    h.write(data)
h.close() 

In [28]:
# join each genes into one as one corresponding object.(cDNA except orb1ab)
query_list = ['surface_glycoprotein_query', 'ORF3a_protein_query', 'envelope_protein_query', 'membrane_glycoprotein_query' ,'ORF6_protein_query' ,'ORF7a_protein_query' ,'ORF7b_query', 'ORF8_protein_query', 'nucleocapsid_phosphoprotein_query', 'ORF10_protein_query']            
c_set = {}
for query in query_list:
    dic = read_fasta('../data/%s_cDNA_dict_muscle.fa'%query)
    for i in range(len(dic.keys())):
        key = list(dic.keys())[i]
        if key in c_set.keys():
            c_set[key] = c_set[key] + dic[key]
            #print(c_set[key][:25])
        else:
            c_set[key] = dic[key]



h = open("../data/SARS_CoV_N_cDNA_sequence.muscle.fa", 'w')
for name in c_set.keys():
    data = "> %s\n%s\n" %(name, c_set[name])
    h.write(data)
h.close() 